<a href="https://colab.research.google.com/github/EvGdk/Machine-Learning-HSE/blob/master/%D0%9E%D1%82%D0%BB%D0%B8%D1%87%D0%B8%D1%82%D1%8C_%D1%81%D1%82%D0%B8%D1%85%D0%B8_%D0%BE%D1%82_%D0%BF%D1%80%D0%BE%D0%B7%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip -qq install rusenttokenize --progress-bar off

In [0]:
! pip install pymorphy2

     |████████████████████████████████| 51kB 2.0MB/s 
     |████████████████████████████████| 7.1MB 6.9MB/s 


In [0]:
import pandas as pd
import numpy as np

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [0]:
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from rusenttokenize import ru_sent_tokenize
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()
import pymorphy2
import string

In [0]:
import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

In [0]:
from google.colab import files
files.upload()

In [0]:
!ls

korpus.csv  sample_data


In [0]:
df = pd.read_csv('korpus.csv')

In [0]:
df.tail()

,Unnamed: 0,text,type,author
6440,170,"Это оттого, что с людьми, с которыми она тепер...",prose,pasternak
6441,171,"Доступное, при желаньи, восхищенным пониманиям...",prose,pasternak
6442,172,"Они встречались, крепко спаянные его давностью...",prose,pasternak
6443,173,"Те, которым не пришлось возобновить временно п...",prose,pasternak
6444,174,"Все обзавелись семьями, у всех",prose,pasternak


# Вариант 1. CountVectorizer и MultinomialNB

In [0]:
vec = CountVectorizer()
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.87      0.67      0.76       554
       prose       0.84      0.95      0.89      1058

    accuracy                           0.85      1612
   macro avg       0.86      0.81      0.82      1612
weighted avg       0.85      0.85      0.85      1612



# Вариант 2: CountVectorizer со стоп-словами, MultinomialNB

In [0]:
punctuation = list(punctuation)

In [0]:
punct2 =['—', '«', '»', '–']

In [0]:
punctuation2  =  list(punct2+punctuation)

In [0]:
noise = stopwords.words('russian')
noise_with_punct = stopwords.words('russian') + punctuation2

In [0]:
len(noise)

151

In [0]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words= noise)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test))) #без включения пунктуации в стоп слова

              precision    recall  f1-score   support

       poeme       0.88      0.69      0.78       563
       prose       0.85      0.95      0.90      1049

    accuracy                           0.86      1612
   macro avg       0.87      0.82      0.84      1612
weighted avg       0.86      0.86      0.86      1612



In [0]:
vec = CountVectorizer(tokenizer=word_tokenize, stop_words=noise_with_punct)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


              precision    recall  f1-score   support

       poeme       0.82      0.77      0.79       556
       prose       0.88      0.91      0.90      1056

    accuracy                           0.86      1612
   macro avg       0.85      0.84      0.85      1612
weighted avg       0.86      0.86      0.86      1612



### Веса признаков:

In [0]:
clf.coef_.shape

(1, 27611)

In [0]:
max(clf.coef_[0])

-5.050309155015829

In [0]:
print(max(enumerate(clf.coef_[0]), key=lambda pair: pair[1]))

(6044, -5.050309155015829)


In [0]:
index_to_word = {
    ind: word
    for (word, ind)
    in vec.vocabulary_.items()
    }

In [0]:
index_to_word[6044]

'её'

In [0]:
top_features = sorted(enumerate(clf.coef_[0]), key=lambda pair: pair[1], reverse=True)[:20]
for index, value in top_features:
  print(index_to_word[index])

её
это
ещё
всё
очень
сказал
глаза
сказала
неё
что-то
стал
возле
своей
руку
натали
руки
день
ко
стала
которой


## Проверим классификатор

In [0]:
vector1 = vec.transform([""" Духовной жаждою томим,
В пустыне мрачной я влачился, —
И шестикрылый серафим
На перепутье мне явился.
Перстами легкими как сон
Моих зениц коснулся он.
Отверзлись вещие зеницы,
Как у испуганной орлицы.
Моих ушей коснулся он, —
И их наполнил шум и звон:
И внял я неба содроганье,
И горний ангелов полет,
И гад морских подводный ход,
И дольней лозы прозябанье."""])

In [0]:
clf.predict(vector1) #Пушкин "Пророк"

array(['poeme'], dtype='<U5')

In [0]:
vector2 = vec.transform([""" Однажды играли в карты у конногвардейца Нарумова. Долгая зимняя ночь прошла незаметно; сели ужинать в пятом часу утра. Те, которые остались в выигрыше, ели с большим апетитом; прочие, в рассеянности, сидели перед пустыми своими приборами. Но шампанское явилось, разговор оживился, и все приняли в нем участие.

-- Что ты сделал, Сурин? -- спросил хозяин.

-- Проиграл, по обыкновению. Надобно признаться, что я несчастлив: играю мирандолем, никогда не горячусь, ничем меня с толку не собьешь, а всё проигрываюсь!"""])

In [0]:
clf.predict(vector2) #Это Пушкин, "Пиковая дама"

array(['prose'], dtype='<U5')

In [0]:
vector3 = vec.transform([""" …Я бы хотела жить с Вами
В маленьком городе,
Где вечные сумерки
И вечные колокола.
И в маленькой деревенской гостинице —
Тонкий звон
Старинных часов — как капельки времени.
И иногда, по вечерам, из какой-нибудь мансарды —
Флейта,
И сам флейтист в окне.
И большие тюльпаны на окнах.
И может быть, Вы бы даже меня не любили…

Посреди комнаты — огромная изразцовая печка,
На каждом изразце — картинка:
Роза — сердце — корабль. —
А в единственном окне —
Снег, снег, снег.

Вы бы лежали — каким я Вас люблю: ленивый,
Равнодушный, беспечный.
Изредка резкий треск
Спички.

Папироса горит и гаснет,
И долго-долго дрожит на ее краю
Серым коротким столбиком — пепел.
Вам даже лень его стряхивать —
И вся папироса летит в огонь."""])

In [0]:
clf.predict(vector3) # А вот белый стих Марины Цветаевой

array(['prose'], dtype='<U5')

In [0]:
vector4 = vec.transform([""" Она пришла с мороза,
Раскрасневшаяся,
Наполнила комнату
Ароматом воздуха и духов,
Звонким голосом
И совсем неуважительной к занятиям
Болтовней.
Она немедленно уронила на пол
Толстый том художественного журнала,
И сейчас же стало казаться,
Что в моей большой комнате
Очень мало места.
Всё это было немножко досадно
И довольно нелепо.
Впрочем, она захотела,
Чтобы я читал ей вслух «Макбета».
Едва дойдя до пузырей земли,
О которых я не могу говорить без волнения,
Я заметил, что она тоже волнуется
И внимательно смотрит в окно.
Оказалось, что большой пестрый кот
С трудом лепится по краю крыши,
Подстерегая целующихся голубей.
Я рассердился больше всего на то,
Что целовались не мы, а голуби,
И что прошли времена Паоло и Франчески."""])

In [0]:
clf.predict(vector4) # Еще один белый стих. Александр Блок

array(['prose'], dtype='<U5')

In [0]:
vector5 = vec.transform(["""Скоро Пасха. Наклонится Бог
и дохнёт осторожно и жарко
на убогие веточки в парке,
на клубнику и чертополох.

На крапиву и розовый куст,
на живое и на неживое.
И луга защекочет травою,
и у ветра изменится вкус.

По пустым одичалым лесам
разлетится далёкое эхо
то ли возгласа, а то ли смеха,
тихий шёпот, шаги, голоса.

И от горсточки сосен кривых,
замордованных ближней деревней,
вдруг пахнёт настоящим и древним,
заставляющим прыгать и выть. """])

In [0]:
clf.predict(vector5)

array(['poeme'], dtype='<U5')

In [0]:
vector6 = vec.transform([""" Мертвенный пепел лун в трауре неба,
Перхотью буквы звезд - мое имя,
Чтобы его прочесть столько верст.

Нибелунг, ничего у тебя не выйдет -
кошка сдохла, хвост облез.
И никто эту кровь нe выпьет, и никто
ее плоть не съест.

Ждешь? Врешь! В руках синдромная
дрожь. Пьешь? Что ж...
На то и солнечный день раскис
в квадрате окна.
И твоя мама больна, и твоя мама одна.
Утешься собственным сном, где я -
рябиной за окном.

Вольному руки греть в пламени танго,
Я заклинаю пить воды Конго,
Чтобы пожар отмыть - петь да петь.

Нибелунг, это палит костры туземка -
бронзовая самка гну.
И ты в клетке ее так крепко, что
не поймешь, почему...

"Весна" - похмельный сладко мурчит бес сна.
Вчера была тарида, сегодня в горле блесна.
Твое вино не беда, когда вина не ясна.
Еще одним серым днем на кухне с грязным столом,
Где я - рябиной за окном.

В памяти млечных рун - смерти и корни.
В рунах движения зла в миокарде,
Чтобы его простить - два крыла.

Нибелунг, это плавит твой воск конвектор -
Перья крыльев вмерзли в сталь.
Память в трубы уносит ветром... Улетай! Улетай!

Семь бед - один ответ - Бога нет как нет,
Где на столе будет гроб, там на столе будет спирт.
Где за столом кто-то пьет, там под столом кто-то спит.
Где человеческий лом присыпан хлоркой и льдом -
Там я - рябиной за окном."""])

In [0]:
clf.predict(vector6)

array(['poeme'], dtype='<U5')

# Вариант 3: CountVectorizer, NB, pymorphy

In [0]:
morph_analyzer = pymorphy2.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

In [0]:
def preprocess_tokenize(text):
    
    text_preprocessed_tokenized = []
        
    for sentence in ru_sent_tokenize(text):
        clean_words = [word.strip(string.punctuation) for word in word_tokenize(text)]
        clean_words = [word for word in clean_words if word]
        clean_words = [word.lower() for word in clean_words if word]
        clean_words = [word for word in clean_words if word not in russian_stopwords]
        clean_lemmas = [morph_analyzer.parse(word)[0].normal_form for word in clean_words]
        text_preprocessed_tokenized.extend(clean_lemmas)

    return text_preprocessed_tokenized

In [0]:
vec = CountVectorizer(tokenizer=preprocess_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)
print(classification_report(y_test, nb.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.87      0.72      0.79       540
       prose       0.87      0.95      0.91      1072

    accuracy                           0.87      1612
   macro avg       0.87      0.83      0.85      1612
weighted avg       0.87      0.87      0.87      1612



### Веса признаков:

In [0]:
clf.coef_.shape

(1, 14430)

In [0]:
max(clf.coef_[0])

-3.3300817844282466

In [0]:
print(max(enumerate(clf.coef_[0]), key=lambda pair: pair[1]))

(14423, -3.3300817844282466)


In [0]:
index_to_word = {
    ind: word
    for (word, ind)
    in vec.vocabulary_.items()
    }

In [0]:
index_to_word[14423]

'—'

In [0]:
top_features = sorted(enumerate(clf.coef_[0]), key=lambda pair: pair[1], reverse=True)[:10]
for index, value in top_features:
  print(index_to_word[index])

—
её
весь
это
«
»
сказать
рука
глаз
свой


# Вариант 4: LogisticRegression

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
vec = CountVectorizer(tokenizer=preprocess_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.78      0.79      0.78       589
       prose       0.88      0.87      0.87      1023

    accuracy                           0.84      1612
   macro avg       0.83      0.83      0.83      1612
weighted avg       0.84      0.84      0.84      1612



очень плохо(((( Попробуем без pymorphy:

In [0]:
vec = CountVectorizer(tokenizer=word_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.79      0.82      0.80       541
       prose       0.91      0.89      0.90      1071

    accuracy                           0.86      1612
   macro avg       0.85      0.85      0.85      1612
weighted avg       0.87      0.86      0.87      1612



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Теперь то же самое со стоп-словами (с пунктуацией)


In [0]:
vec = CountVectorizer(tokenizer=word_tokenize,stop_words=noise_with_punct )
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


              precision    recall  f1-score   support

       poeme       0.88      0.54      0.67       552
       prose       0.80      0.96      0.87      1060

    accuracy                           0.82      1612
   macro avg       0.84      0.75      0.77      1612
weighted avg       0.83      0.82      0.80      1612



Стоп - слова без пунктуации:

In [0]:
vec = CountVectorizer(tokenizer=word_tokenize,stop_words=noise )
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.83      0.76      0.79       591
       prose       0.87      0.91      0.89      1021

    accuracy                           0.85      1612
   macro avg       0.85      0.83      0.84      1612
weighted avg       0.85      0.85      0.85      1612



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Вариант 5: CountVectorizer, LogisticRegression, Bi-gramms

In [0]:
vec = CountVectorizer(ngram_range=(1, 3)) 
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
lr = LogisticRegression()
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.81      0.58      0.67       571
       prose       0.80      0.93      0.86      1041

    accuracy                           0.80      1612
   macro avg       0.81      0.75      0.77      1612
weighted avg       0.80      0.80      0.79      1612



## GridSearch

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
param_grid = {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']
    }

In [0]:
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values)
grid_clf_acc.fit(X_train, y_train)

In [0]:
print("tuned hpyerparameters :(best parameters) ",grid_clf_acc.best_params_)
print("accuracy :", grid_clf_acc.best_score_)

In [0]:
vectorizer = CountVectorizer(tokenizer=preprocess_tokenize, stop_words=noise_with_punct)
bow = vectorizer.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bow, df.type)
lr = LogisticRegression(C=25, penalty='l2', solver = 'saga')
clf = lr.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.82      0.67      0.74       571
       prose       0.84      0.92      0.88      1041

    accuracy                           0.83      1612
   macro avg       0.83      0.80      0.81      1612
weighted avg       0.83      0.83      0.83      1612



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## Прочие классификаторы

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

### RandomForestClassifier

In [0]:
vec = CountVectorizer(tokenizer=preprocess_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
rf = RandomForestClassifier()
clf = rf.fit(X_train, y_train)

In [0]:
print(classification_report(y_test, rf.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.76      0.63      0.69       541
       prose       0.83      0.90      0.86      1071

    accuracy                           0.81      1612
   macro avg       0.80      0.77      0.78      1612
weighted avg       0.81      0.81      0.81      1612



### KNeighborsClassifier

In [0]:
vec = CountVectorizer(tokenizer=preprocess_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
kn = KNeighborsClassifier()
clf = kn.fit(X_train, y_train)

In [0]:
print(classification_report(y_test, kn.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.58      0.41      0.48       541
       prose       0.74      0.85      0.79      1071

    accuracy                           0.70      1612
   macro avg       0.66      0.63      0.64      1612
weighted avg       0.69      0.70      0.69      1612



### SVC

In [0]:
vec = CountVectorizer(tokenizer=preprocess_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
svc = SVC()
clf = svc.fit(X_train, y_train)

In [0]:
print(classification_report(y_test, svc.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.73      0.77      0.75       575
       prose       0.87      0.84      0.86      1037

    accuracy                           0.82      1612
   macro avg       0.80      0.81      0.80      1612
weighted avg       0.82      0.82      0.82      1612



### DecisionTreeClassifier

In [0]:
vec = CountVectorizer(tokenizer=preprocess_tokenize)
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.type)
dt = DecisionTreeClassifier()
clf = dt.fit(X_train, y_train)

In [0]:
print(classification_report(y_test, dt.predict(X_test)))

              precision    recall  f1-score   support

       poeme       0.77      0.50      0.61       586
       prose       0.76      0.92      0.83      1026

    accuracy                           0.77      1612
   macro avg       0.77      0.71      0.72      1612
weighted avg       0.77      0.77      0.75      1612

